In [3]:
# -------------------------------
# 1. Install libraries
# -------------------------------
# !pip install transformers datasets seqeval

# -------------------------------
# 2. Imports
# -------------------------------
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification
import evaluate
import numpy as np

# -------------------------------
# 3. Parse CoNLL file into dataset
# -------------------------------
def parse_conll(file_path):
    sentences = []
    tokens = []
    labels = []

    with open(file_path, encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line:
                if tokens:
                    sentences.append({'tokens': tokens, 'ner_tags': labels})
                    tokens = []
                    labels = []
            else:
                splits = line.split()
                if len(splits) >= 2:
                    tokens.append(splits[0])
                    labels.append(splits[1])
        # add last sentence
        if tokens:
            sentences.append({'tokens': tokens, 'ner_tags': labels})
    return sentences

# Replace with your labeled dataset path
data = parse_conll("amharic_ner_conll_labeled_2.txt")

# Check if dataset is empty
if len(data) == 0:
    raise ValueError("Parsed dataset is empty. Check your CoNLL file formatting.")

# Build label mapping
all_labels = sorted(list({label for sent in data for label in sent['ner_tags']}))
label2id = {l: i for i, l in enumerate(all_labels)}
id2label = {i: l for i, l in enumerate(all_labels)}

# Map string labels to integers
for sent in data:
    sent['ner_tags'] = [label2id[label] for label in sent['ner_tags']]

# Create Hugging Face dataset
dataset = Dataset.from_list(data)
dataset = dataset.train_test_split(test_size=0.1)

# -------------------------------
# 4. Models to compare
# -------------------------------
model_names = {
    "XLM-Roberta": "xlm-roberta-base",
    "DistilBERT": "distilbert-base-multilingual-cased",
    "mBERT": "bert-base-multilingual-cased"
}

# -------------------------------
# 5. Tokenization & alignment
# -------------------------------
def tokenize_and_align_labels(batch, tokenizer):
    tokenized_inputs = tokenizer(batch["tokens"], truncation=True, is_split_into_words=True)
    aligned_labels = []
    for i, label in enumerate(batch["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                # Inside subword: keep I- labels only
                current_label = label[word_idx]
                label_name = id2label[current_label]
                label_ids.append(current_label if label_name.startswith('I-') else -100)
            previous_word_idx = word_idx
        aligned_labels.append(label_ids)
    tokenized_inputs["labels"] = aligned_labels
    return tokenized_inputs

# -------------------------------
# 6. Evaluation Metrics
# -------------------------------
seqeval = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=-1)
    true_labels = [
        [id2label[l] for l in label if l != -100]
        for label in labels
    ]
    true_predictions = [
        [id2label[pred] for (pred, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    
    # Avoid undefined metrics by filling zero_division
    results = seqeval.compute(predictions=true_predictions, references=true_labels, zero_division=0)
    return {
        "precision": results.get("overall_precision", 0),
        "recall": results.get("overall_recall", 0),
        "f1": results.get("overall_f1", 0),
        "accuracy": results.get("overall_accuracy", 0)
    }

# -------------------------------
# 7. Loop through models
# -------------------------------
results = {}

for name, model_name in model_names.items():
    print(f"\n=== Fine-tuning {name} ({model_name}) ===")
    
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForTokenClassification.from_pretrained(
        model_name,
        num_labels=len(label2id),
        id2label=id2label,
        label2id=label2id
    )

    tokenized_dataset = dataset.map(lambda x: tokenize_and_align_labels(x, tokenizer), batched=True)

    data_collator = DataCollatorForTokenClassification(tokenizer)

    training_args = TrainingArguments(
        output_dir=f"./{name}_ner_model",
        evaluation_strategy="epoch",
        learning_rate=5e-5,
        per_device_train_batch_size=2,  # reduce for CPU or GPU memory
        per_device_eval_batch_size=2,
        num_train_epochs=3,
        weight_decay=0.01,
        save_total_limit=1,
        logging_dir=f'./logs_{name}',
        logging_steps=10
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset["train"],
        eval_dataset=tokenized_dataset["test"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics
    )

    trainer.train()
    metrics = trainer.evaluate()
    results[name] = metrics
    trainer.save_model(f"./{name}_ner_model_final")
    tokenizer.save_pretrained(f"./{name}_ner_model_final")

# -------------------------------
# 8. Compare results (fixed)
# -------------------------------
for model, metrics in results.items():
    print(f"\nModel: {model}")
    print(f"Precision: {metrics.get('eval_precision', 0):.3f}")
    print(f"Recall:    {metrics.get('eval_recall', 0):.3f}")
    print(f"F1-score:  {metrics.get('eval_f1', 0):.3f}")
    print(f"Accuracy:  {metrics.get('eval_accuracy', 0):.3f}")




=== Fine-tuning XLM-Roberta (xlm-roberta-base) ===


Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 5/5 [00:00<00:00, 714.46 examples/s]
c:\Users\pc\Desktop\10 Academy\Week 4\E-commerce\venv\Lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  0%|          | 0/69 [00:00<?, ?it/s]c:\Users\pc\Desktop\10 Academy\Week 4\E-commerce\venv\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
 14%|█▍        | 10/69 [00:26<02:06,  2.14s/it]

{'loss': 1.2556, 'grad_norm': 2.745151996612549, 'learning_rate': 4.27536231884058e-05, 'epoch': 0.43}


 29%|██▉       | 20/69 [00:45<01:36,  1.96s/it]

{'loss': 0.7006, 'grad_norm': 11.66637134552002, 'learning_rate': 3.5507246376811596e-05, 'epoch': 0.87}


...

{'loss': 0.245, 'grad_norm': 6.0889387130737305, 'learning_rate': 6.521739130434783e-06, 'epoch': 2.61}


100%|██████████| 69/69 [01:36<00:00,  1.32s/it]c:\Users\pc\Desktop\10 Academy\Week 4\E-commerce\venv\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)

100%|██████████| 69/69 [01:39<00:00,  1.44s/it]


{'eval_loss': 0.11059578508138657, 'eval_precision': 0.9090909090909091, 'eval_recall': 0.6666666666666666, 'eval_f1': 0.7692307692307692, 'eval_accuracy': 0.9727272727272728, 'eval_runtime': 0.327, 'eval_samples_per_second': 15.291, 'eval_steps_per_second': 9.174, 'epoch': 3.0}
{'train_runtime': 99.2303, 'train_samples_per_second': 1.36, 'train_steps_per_second': 0.695, 'train_loss': 0.41962648992953094, 'epoch': 3.0}


100%|██████████| 3/3 [00:00<00:00, 13.57it/s]



Model: XLM-Roberta
Precision: 0.450
Recall:    0.600
F1-score:  0.514
Accuracy:  0.968

Model: DistilBERT
Precision: 0.750
Recall:    0.400
F1-score:  0.522
Accuracy:  0.936

Model: mBERT
Precision: 0.909
Recall:    0.667
F1-score:  0.769
Accuracy:  0.973
